In [59]:
# Import all necesary packages and libraries 
from json import JSONDecodeError
import requests
from textblob import TextBlob
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np

In [60]:
# Replace with your own values
access_token = 'EAAMC8ERtLFABOxSzoa6lUm6CsjhNA63cb5qQOIzV4dgCaflZCr8i4ovkB0tkeZCd3AUDs0wA9LN4ZBUS5DG8ipMlApdnZCbkc7j6C9RokqY28xGAvFPLMbZBZBXlJ7lyQoAe7sWIpgv7kb9KBzvgd4qmwbNToL2O6bZCnmj57XZC6btJ6pCnrh4cBmbZC7jDgX1nkghOWnE6tMTqIJpfBOC26F422b6Cwboxsg7h1SBd1ph6S'
instagram_account_id = '17841406879294936'

# API endpoint URLs
base_url = 'https://graph.facebook.com/v17.0/'
media_url = f'{base_url}{instagram_account_id}/media'
insights_url = f'{base_url}{instagram_account_id}/insights'

# Parameters for the API requests
media_params = {
    'fields': 'id,media_type,media_url,caption,timestamp,comments{comment_count,replies}',
    'access_token': access_token
}

insights_params = {
    'metric': 'likes,comments',
    'metric_type': 'total_value',
    'period': 'day',
    'access_token': access_token
}

# Function to perform sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity # type: ignore
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

# Function to fetch and print media data
def get_media_data():
    response = requests.get(media_url, params=media_params)
    print("Response Content:", response.text)
    try:
        data = response.json()
        if 'data' in data:
            for media in data['data']:
                print(f"Media ID: {media['id']}")
                print(f"Media Type: {media['media_type']}")
                print(f"Media URL: {media['media_url']}")
                print(f"Caption: {media.get('caption', 'No caption')}")
                print(f"Timestamp: {media['timestamp']}")
                print(f"Comments: {media.get('comments', 'No Comments')}")
                print("--------------------------")
        else:
            print("No media data available.")

    except JSONDecodeError as e:
        print("JSON Decode Error:", e)
        print("Response Content:", response.text)

# Function to fetch and print insights data
def get_insights_data():
    response = requests.get(insights_url, params=insights_params)
    print("Response Content:", response.text)
    try:
        data = response.json()
        if 'data' in data:
            for insight in data['data']:
                if 'period_start' in insight:
                    print(f"Date: {insight['period_start']}")
                    for metric, value in insight['values'][0].items():
                        print(f"{metric.capitalize()}: {value}")
                    print("--------------------------")
                #else:
                 #   print("Error: 'period_start' key not found in insight data")
                print()
        else:
            print("No media data available.")

    except JSONDecodeError as e:
        print("JSON Decode Error:", e)
        print("Response Content:", response.text)

if __name__ == "__main__":
    print("Media Data:")
    get_media_data()

    print("\nInsights Data:")
    get_insights_data()


Media Data:
Response Content: {"data":[{"id":"18013314199769788","media_type":"IMAGE","media_url":"https:\/\/scontent.cdninstagram.com\/v\/t51.29350-15\/369169963_944690173273976_164820808347549626_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=3O26DLnUeJIAX-lfs6u&_nc_ht=scontent.cdninstagram.com&edm=AM6HXa8EAAAA&oh=00_AfC7v52RECsqk0mc-vjHo8bxBb7sPJLiMb6Y_Q3x8BWerg&oe=64E9FE3C","caption":"Information is only powerful once accessed.","timestamp":"2023-08-21T13:45:44+0000","comments":{"data":[{"id":"17991319139476413"},{"id":"18085462810372683"}]}},{"id":"17871242314788623","media_type":"IMAGE","media_url":"https:\/\/scontent.cdninstagram.com\/v\/t51.29350-15\/105990660_3002047319832981_3547706877008675002_n.jpg?_nc_cat=106&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=gsfWsmVrC7IAX_1Hvdi&_nc_ht=scontent.cdninstagram.com&edm=AM6HXa8EAAAA&oh=00_AfATK1aaLXMlmHmz0nrHK-iEhYlfcP93lw7eQA_jscb8Pg&oe=64E8CCDF","caption":"Keep pushing","timestamp":"2020-06-28T05:40:31+0000","comments":{"data":[{"replies":{"da